In [1]:
import os
import io
import sys
import matplotlib.pyplot as plt
import IPython.display as ipd
import pandas as pd
import re
import subprocess
import numpy as np
from tqdm import tqdm

%load_ext autoreload
%autoreload 2
%matplotlib inline

sys.path.append('../src')

In [46]:
import re
import ast

logs_root = '/media/maxim/WesternDigital/RAMAS2024/sc_meld/xlstm/'

def parse_config(str_training_config):
    str_training_config = re.sub(' +', ' ', str_training_config.replace('<class ', '').replace('>', ''))
    training_config = ast.literal_eval(str_training_config)
    return (training_config['FEATURE_EXTRACTOR']['WIN_MAX_LENGTH'], training_config['FEATURE_EXTRACTOR']['WIN_SHIFT'], 
            training_config['FEATURE_EXTRACTOR']['cls'].split('.')[-1], training_config['MODEL']['cls'].split('.')[-1])



ds_name = 'devel'
ds_name = 'test'

measure_name = 'emo_sen_combined'
measure_name = 'emo_UAR'

res = []
for exp in sorted(os.listdir(logs_root)):
    csv_stats = pd.read_csv(os.path.join(logs_root, exp, 'logs', 'stats.csv'), sep=';')

    best_model = csv_stats.iloc[csv_stats['{0}_{1}'.format(ds_name, measure_name)].idxmax()].to_dict()
    with open(os.path.join(logs_root, exp, 'logs', 'source.log')) as file:
        source_lines = [line.rstrip() for line in file]

    exp_params = parse_config(''.join(source_lines[17:34]))
    d_res = {'exp_name': exp, 'win_max_len': exp_params[0], 'win_shift': exp_params[1], 'feature_extractor': exp_params[2], 'model': exp_params[3]} | best_model
    res.append(d_res)

res_df = pd.DataFrame.from_dict(res)
res_df.to_csv('MELD_xlstm_results.csv')